In [109]:
#import libraries

import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import hvplot.pandas
import hvplot
#import matplotlib
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
from bokeh.plotting import show
#import json

In [110]:
#load .env file

load_dotenv()

True

In [111]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [112]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [113]:
# Request stock symbol from user. This code will work will all available tickers on alpacas.
# "META" ticker has been temporarily hard coded into the system for ease of use

# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")

stock_symbol = "AAPL"

In [114]:
# Establish what time frame increment we will be looking at
# currently we are working with the daily chart, but the theory would be to 
# allow the user to identify what time scale they would like to trade on
# shorter time frame allows for "day trades" larger time scales are geared
# more towards longer term investments and swing trading.

time_frame = "1Day"

In [115]:
# Get user input for the start date in datetime format
# the start date has been hard coded for ease of programming and troubleshooting our code

# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2016-01-01", tz="America/New_York").isoformat()

In [116]:
# Get user input for the end date in datetime format
# the end date has been hard coded for ease of programming and troubleshooting our code

# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

In [117]:
# trading_days = end_date - start_date

# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [118]:
# Calls the stock data from the period of time desired in the desired time increment

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [119]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2016-01-04 05:00:00+00:00 to 2019-12-31 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        1006 non-null   float64
 1   high         1006 non-null   float64
 2   low          1006 non-null   float64
 3   trade_count  1006 non-null   int64  
 4   open         1006 non-null   float64
 5   volume       1006 non-null   int64  
 6   vwap         1006 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 62.9 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,351452,102.61,71935339,104.091749
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,321365,105.75,58690536,103.400868
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,409164,100.56,71079827,100.843959
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,462836,98.68,85996453,98.011371
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,419555,98.55,75066292,97.897369


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2019-12-24 05:00:00+00:00,284.27,284.89,282.9197,82671,284.69,12765748,284.098909
2019-12-26 05:00:00+00:00,289.91,289.98,284.7000,170219,284.82,24929336,288.220391
2019-12-27 05:00:00+00:00,289.80,293.97,288.1200,263248,291.12,39190841,290.721721
2019-12-30 05:00:00+00:00,291.52,292.69,285.2200,264189,289.46,37709308,289.980227
2019-12-31 05:00:00+00:00,293.65,293.68,289.5200,185422,289.93,27817964,292.471404


In [120]:
# Reset index and display the first 5
# this also prepares for use with plotly library/visualization

#stock_data_plot = stock_data.reset_index()

# clean up datafram by removing vwap, volume, trade count columns
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results

stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
stock_data_plot = stock_data.reset_index()

stock_data.head()

,close,high,low,open
timestamp,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55


In [121]:
# Assign plot values to plotly

fig = go.Candlestick(x=stock_data_plot['timestamp'],
                open=stock_data_plot['open'],
                high=stock_data_plot['high'],
                low=stock_data_plot['low'],
                close=stock_data_plot['close'])


cand = go.Figure(data=[fig])

In [122]:
# Customize the figure (optional)

cand.update_layout(
    width=1200, height=800,
    title="AAPL, OCT - DEC",
    yaxis_title='AAPL Stock'
)


In [123]:
# Add new data columns to DF for body size and candle size
# body size and direction is value of open - close. 
# candle size is the absolute value of low - high

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

# Display columns with added columns

stock_data.head()

,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61,2.74,3.368
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75,-3.04,3.440
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56,0.14,2.500
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68,-2.23,3.700
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55,-1.59,2.350


In [124]:
# initialize variables to be used in calculations
# identify the max body size and minimum candle size within the dataframe

max_body_size = stock_data["body_size"].max()
min_candle_size = stock_data["candle_size"].min()

# keeps track of the count which would initiate a buy or sell
# current count is initialized at 0 as the count is calculated based off of historical data

current_count = 0

# the threshold that would initiate a buy and or sell

threshold = 2

# adds a documenting column to the dataframe and assigns a flat value to all rows

stock_data["card_count"] = current_count

# identifies the number of rows to be used to break the for loop

number_of_rows = len(stock_data)

# print information up to date to identify data progress

# print(max_body_size)
# print(min_candle_size)
# print(previous_candle)
# print(stock_data.iloc[0][4])
# print(number_of_rows)
# stock_data.head()

In [125]:
# ...::: working code :::...
# initialize variables and dataframe for tracking buys, sells, and profits

initial_investment = 10000

# initialize strategy tracking variables and add to stock_data df

stock_data["capital"] = initial_investment
stock_data["number_of_shares"] = 0
stock_data["investment"] = 0
stock_data["profits"] = 0

# variables that will be used to calculate profits/losses from buys/sells

capital = initial_investment
number_of_shares = 0
investment = 0
profits = 0

stock_data.head()

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61,2.74,3.368,0,10000,0,0,0
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75,-3.04,3.440,0,10000,0,0,0
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56,0.14,2.500,0,10000,0,0,0
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68,-2.23,3.700,0,10000,0,0,0
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55,-1.59,2.350,0,10000,0,0,0


In [126]:
# initialize variables and assign values based off of first row for calculations

previous_close = stock_data.iloc[0][0]
previous_high = stock_data.iloc[0][1]
previous_low = stock_data.iloc[0][2]
previous_open = stock_data.iloc[0][3]
previous_body_size = stock_data.iloc[0][4]
previous_candle_size = stock_data.iloc[0][5]

# begin for loop to start calculating each row's data to assign a +1 or -1
# to our count and add adjust our current/previous counts accordingly

for index, row in islice(stock_data.iterrows(), 1, None):

# create an if statement to make sure the for loop finishes at the end of the data appropriately  
    
    if index == (number_of_rows):
        break
    
# assign current day numbers to work with conditional statements

    current_close = row[0]
    current_high = row[1]
    current_low = row[2]
    current_open = row[3]
    current_body_size = row[4]
    current_candle_size = row[5]
    # capital = row[7]
    # number_of_shares = row[8]
    # investment = row[9]
    # profits = row[10]

   # begin of conditional statements. this portion identifies what values to assign
   # to each individal row's data and to determine whether a threshold of "sell" or "buy"
   # is reached. this portion can be adjusted/added to in order to take into consideration
   # additional bar patterns

    # bullish hammer
    if (current_body_size > 0) and (current_body_size*2) < (current_candle_size):
        current_count = current_count + 1
    # bearish hammer
    elif (current_body_size < 0) and ((np.absolute(current_body_size)*2) < (current_candle_size)):
        current_count = current_count - 1
    # bullish engulfing candle
    elif (current_body_size > 0) and (previous_body_size < 1) and (current_body_size > previous_body_size):
        current_count = current_count + 1
    # bearish engulfing candle
    elif (current_body_size < 0) and (previous_body_size > 1) and (np.absolute(current_body_size) > (previous_body_size)):
        current_count = current_count - 1
    # bullish doji candle
    #elif (previous_body_size < 0) and (np.absolute(current_body_size) < (current_close * max_body_size)):
    #    current_count = current_count + 1

    # contains the current count actual value to the desired "buy/sell" threshold

    if current_count > threshold:
        current_count = threshold
    elif current_count < -(threshold):
        current_count = -(threshold)

    # conditional statements to identify when to purchase or sell a stock

    if (current_count == threshold) and (number_of_shares == 0):
        number_of_shares = np.floor(capital / current_close)
        investment = number_of_shares * current_close
        capital = capital - investment
    elif (current_count == -threshold) and (number_of_shares != 0):
        profits = (number_of_shares * current_close) - investment
        capital = capital + investment + profits
        investment = 0
        number_of_shares = 0
    else:
        profits = (number_of_shares * current_close) - investment

        #stock_data.at[index,"investment"] = number_of_shares * current_close
        #profits = row[9] - investment
        #stock_data.at[index,"profits"] = profits

    stock_data.at[index,"number_of_shares"] = number_of_shares
    stock_data.at[index,"investment"] = investment
    stock_data.at[index,"profits"] = profits
    stock_data.at[index,"capital"] = capital
    # set variables to prepare to analyze the following row

    previous_close = row[0]
    previous_high = row[1]
    previous_low = row[2]
    previous_open = row[3]
    previous_body_size = row[4]
    previous_candle_size = row[5]

    # keep track of the current count within the dataframe. assign that
    # to the current working row
    
    stock_data.at[index,"card_count"] = current_count

if (number_of_shares == 0):
    total_profit = capital
elif (number_of_shares != 0):
    total_profit = (capital + investment + profits) - initial_investment

total_profit = round(total_profit,2)
percent_profit = (total_profit/initial_investment)*100

print(f"total profits for your autotrader are: ${total_profit} a {percent_profit}%")
    
display(stock_data.head(40))
display(stock_data.tail(40))





total profits for your autotrader are: $14973.34 a 149.7334%


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.3680,102.0000,102.61,2.74,3.3680,0,10000.0,0,0.0,0.0
2016-01-05 05:00:00+00:00,102.71,105.8500,102.4100,105.75,-3.04,3.4400,-1,10000.0,0,0.0,0.0
2016-01-06 05:00:00+00:00,100.70,102.3700,99.8700,100.56,0.14,2.5000,0,10000.0,0,0.0,0.0
2016-01-07 05:00:00+00:00,96.45,100.1300,96.4300,98.68,-2.23,3.7000,0,10000.0,0,0.0,0.0
2016-01-08 05:00:00+00:00,96.96,99.1100,96.7600,98.55,-1.59,2.3500,0,10000.0,0,0.0,0.0
2016-01-11 05:00:00+00:00,98.53,99.0600,97.3400,98.97,-0.44,1.7200,-1,10000.0,0,0.0,0.0
2016-01-12 05:00:00+00:00,99.96,100.6900,98.8399,100.55,-0.59,1.8501,-2,10000.0,0,0.0,0.0
2016-01-13 05:00:00+00:00,97.39,101.1900,97.3000,100.32,-2.93,3.8900,-2,10000.0,0,0.0,0.0
2016-01-14 05:00:00+00:00,99.52,100.4800,95.7400,97.96,1.56,4.7400,-1,10000.0,0,0.0,0.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits
timestamp,,,,,,,,,,,
2019-11-04 05:00:00+00:00,257.50,257.8450,255.3800,257.33,0.17,2.4650,2,13.086,85,18128.8,3758.70
2019-11-05 05:00:00+00:00,257.13,258.1900,256.3200,257.05,0.08,1.8700,2,13.086,85,18128.8,3727.25
2019-11-06 05:00:00+00:00,257.24,257.4900,255.3650,256.77,0.47,2.1250,2,13.086,85,18128.8,3736.60
2019-11-07 05:00:00+00:00,259.43,260.3500,258.1100,258.74,0.69,2.2400,2,13.086,85,18128.8,3922.75
2019-11-08 05:00:00+00:00,260.14,260.4400,256.8500,258.69,1.45,3.5900,2,13.086,85,18128.8,3983.10
2019-11-11 05:00:00+00:00,262.20,262.4700,258.2800,258.30,3.90,4.1900,2,13.086,85,18128.8,4158.20
2019-11-12 05:00:00+00:00,261.96,262.7900,260.9200,261.55,0.41,1.8700,2,13.086,85,18128.8,4137.80
2019-11-13 05:00:00+00:00,264.47,264.7800,261.0700,261.13,3.34,3.7100,2,13.086,85,18128.8,4351.15
2019-11-14 05:00:00+00:00,262.64,264.8800,262.1000,263.75,-1.11,2.7800,1,13.086,85,18128.8,4195.60


In [127]:
comp_number_of_shares = np.floor(initial_investment / stock_data.iloc[0][0])
stock_data["comp_profits"] = (comp_number_of_shares * stock_data["close"]) - initial_investment
stock_data["total_profits"] = (stock_data["capital"] + stock_data["investment"] + stock_data["profits"]) - initial_investment
print(comp_number_of_shares)
display(stock_data.head(40))
display(stock_data.tail(40))


94.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits
timestamp,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.3680,102.0000,102.61,2.74,3.3680,0,10000.0,0,0.0,0.0,-97.10,0.0
2016-01-05 05:00:00+00:00,102.71,105.8500,102.4100,105.75,-3.04,3.4400,-1,10000.0,0,0.0,0.0,-345.26,0.0
2016-01-06 05:00:00+00:00,100.70,102.3700,99.8700,100.56,0.14,2.5000,0,10000.0,0,0.0,0.0,-534.20,0.0
2016-01-07 05:00:00+00:00,96.45,100.1300,96.4300,98.68,-2.23,3.7000,0,10000.0,0,0.0,0.0,-933.70,0.0
2016-01-08 05:00:00+00:00,96.96,99.1100,96.7600,98.55,-1.59,2.3500,0,10000.0,0,0.0,0.0,-885.76,0.0
2016-01-11 05:00:00+00:00,98.53,99.0600,97.3400,98.97,-0.44,1.7200,-1,10000.0,0,0.0,0.0,-738.18,0.0
2016-01-12 05:00:00+00:00,99.96,100.6900,98.8399,100.55,-0.59,1.8501,-2,10000.0,0,0.0,0.0,-603.76,0.0
2016-01-13 05:00:00+00:00,97.39,101.1900,97.3000,100.32,-2.93,3.8900,-2,10000.0,0,0.0,0.0,-845.34,0.0
2016-01-14 05:00:00+00:00,99.52,100.4800,95.7400,97.96,1.56,4.7400,-1,10000.0,0,0.0,0.0,-645.12,0.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits
timestamp,,,,,,,,,,,,,
2019-11-04 05:00:00+00:00,257.50,257.8450,255.3800,257.33,0.17,2.4650,2,13.086,85,18128.8,3758.70,14205.00,11900.586
2019-11-05 05:00:00+00:00,257.13,258.1900,256.3200,257.05,0.08,1.8700,2,13.086,85,18128.8,3727.25,14170.22,11869.136
2019-11-06 05:00:00+00:00,257.24,257.4900,255.3650,256.77,0.47,2.1250,2,13.086,85,18128.8,3736.60,14180.56,11878.486
2019-11-07 05:00:00+00:00,259.43,260.3500,258.1100,258.74,0.69,2.2400,2,13.086,85,18128.8,3922.75,14386.42,12064.636
2019-11-08 05:00:00+00:00,260.14,260.4400,256.8500,258.69,1.45,3.5900,2,13.086,85,18128.8,3983.10,14453.16,12124.986
2019-11-11 05:00:00+00:00,262.20,262.4700,258.2800,258.30,3.90,4.1900,2,13.086,85,18128.8,4158.20,14646.80,12300.086
2019-11-12 05:00:00+00:00,261.96,262.7900,260.9200,261.55,0.41,1.8700,2,13.086,85,18128.8,4137.80,14624.24,12279.686
2019-11-13 05:00:00+00:00,264.47,264.7800,261.0700,261.13,3.34,3.7100,2,13.086,85,18128.8,4351.15,14860.18,12493.036
2019-11-14 05:00:00+00:00,262.64,264.8800,262.1000,263.75,-1.11,2.7800,1,13.086,85,18128.8,4195.60,14688.16,12337.486


In [128]:
# Plot the Comparison between Total Profits and Comparative Profits

stock_data.hvplot.line(x="timestamp", y=('total_profits', 'comp_profits'))

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [129]:
# Plot the Number of Shares in Line Graph

stock_data.hvplot.line(x='timestamp', y='number_of_shares')

:Curve   [timestamp]   (number_of_shares)

In [130]:
# Analysis results dataframe

stock_data['port_size'] = stock_data['capital'] + stock_data['profits'] + stock_data['investment']

In [131]:
display(stock_data.head())
display(stock_data.tail())

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,105.35,105.368,102.00,102.61,2.74,3.368,0,10000.0,0,0.0,0.0,-97.10,0.0,10000.0
2016-01-05 05:00:00+00:00,102.71,105.850,102.41,105.75,-3.04,3.440,-1,10000.0,0,0.0,0.0,-345.26,0.0,10000.0
2016-01-06 05:00:00+00:00,100.70,102.370,99.87,100.56,0.14,2.500,0,10000.0,0,0.0,0.0,-534.20,0.0,10000.0
2016-01-07 05:00:00+00:00,96.45,100.130,96.43,98.68,-2.23,3.700,0,10000.0,0,0.0,0.0,-933.70,0.0,10000.0
2016-01-08 05:00:00+00:00,96.96,99.110,96.76,98.55,-1.59,2.350,0,10000.0,0,0.0,0.0,-885.76,0.0,10000.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2019-12-24 05:00:00+00:00,284.27,284.89,282.9197,284.69,-0.42,1.9703,1,13.086,85,18128.8,6034.15,16721.38,14176.036,24176.036
2019-12-26 05:00:00+00:00,289.91,289.98,284.7000,284.82,5.09,5.2800,2,13.086,85,18128.8,6513.55,17251.54,14655.436,24655.436
2019-12-27 05:00:00+00:00,289.80,293.97,288.1200,291.12,-1.32,5.8500,1,13.086,85,18128.8,6504.20,17241.20,14646.086,24646.086
2019-12-30 05:00:00+00:00,291.52,292.69,285.2200,289.46,2.06,7.4700,2,13.086,85,18128.8,6650.40,17402.88,14792.286,24792.286
2019-12-31 05:00:00+00:00,293.65,293.68,289.5200,289.93,3.72,4.1600,2,13.086,85,18128.8,6831.45,17603.10,14973.336,24973.336


In [132]:
#Plot Line Graph for Portfolio Size

stock_data.hvplot.line(x='timestamp', y='port_size')

:Curve   [timestamp]   (port_size)

In [134]:
# Plot a Line Graph for Card Count

stock_data.hvplot.line(x='timestamp', y='card_count')

:Curve   [timestamp]   (card_count)

In [135]:
stock_data.hvplot.scatter(x='timestamp', y='card_count')

:Scatter   [timestamp]   (card_count)